# Fetch Historical Data
Fetches historical data from the Trafiklab API Kollektivtrafikens Datalabb (KoDa).

## Imports

In [7]:
import os
from pathlib import Path
import sys

root_dir = Path().absolute()
# Strip subdirectories if the notebook started in any
if root_dir.parts[-1:] == ('pipeline',):
    root_dir = Path(*root_dir.parts[:-1])
if root_dir.parts[-1:] == ('src',):
    root_dir = Path(*root_dir.parts[:-1])
root_dir = str(root_dir) 

print(f"Root dir: {root_dir}")
os.chdir(root_dir)
print(Path.cwd())

import src.data_utils.filter as filter
import src.data_utils.ingest as ingest

Local environment
Root dir: C:\Users\royli\Desktop\Courses\ID2223_Scalable_Machine_Learning_and_Deep_Learning\Project
C:\Users\royli\Desktop\Courses\ID2223_Scalable_Machine_Learning_and_Deep_Learning\Project
<module 'src' from 'C:\\Users\\royli\\Desktop\\Courses\\ID2223_Scalable_Machine_Learning_and_Deep_Learning\\Project\\src\\__init__.py'>


ModuleNotFoundError: No module named 'ingest'